In [ ]:
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt

paths = {
    "Angry": (1, 'E:/dataset/Tess/OAF_angry/'),
    "Disgust": (2, 'E:/dataset/Tess/OAF_disgust/'),
    "Fear": (3, 'E:/dataset/Tess/OAF_Fear/'),
    "Happy": (4, 'E:/dataset/Tess/OAF_happy/'),
    "Neutral": (5, 'E:/dataset/Tess/OAF_neutral/'),
    "Pleasant Surprise": (6, 'E:/dataset/Tess/OAF_Pleasant_surprise/'),
    "Sad": (7, 'E:/dataset/Tess/OAF_sad/')
}

for emotion, (feature_number, path) in paths.items():
    if os.path.exists(path):
        print(f"{emotion} (Labels {feature_number}):")
        wav_files = [file for file in os.listdir(path) if file.endswith('.wav')]
        for file in wav_files:
            full_path = os.path.join(path, file)
            print(f"  {full_path} -> Labels  {feature_number}")   

    else:
        print(f"{emotion}: Path not found!")

In [2]:
import numpy as np

# Constants
INPUT_NODES = 7       # Number of input features
FILTER_SIZE = 3       # Size of the convolution filter
POOL_SIZE = 2         # Size of the pooling window
OUTPUT_NODES = 7      # Number of product types
LEARNING_RATE = 0.01
EPOCHS = 10           # Reduced for quicker testing

# ReLU activation function
def relu(x):
    return np.maximum(0, x)

# Convolution Layer
class ConvLayer:
    def __init__(self, filter_size, bias=0.5):
        self.filter = np.random.randn(filter_size)  # Random filter initialization
        self.bias = bias

# Fully Connected Layer
class FullyConnectedLayer:
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(input_size, output_size) - 0.5  # Random weight initialization
        self.biases = np.random.randn(output_size) - 0.5               # Random bias initialization

    def forward(self, inputs):
        return np.dot(inputs, self.weights) + self.biases

# Function to apply convolution
def apply_convolution(input_data, conv_layer):
    output_size = len(input_data) - FILTER_SIZE + 1
    output = np.zeros(output_size)
    for i in range(output_size):
        output[i] = np.sum(input_data[i:i + FILTER_SIZE] * conv_layer.filter) + conv_layer.bias
    return relu(output)  # ReLU activation

# Function to apply max pooling
def apply_pooling(input_data, pool_size):
    output_size = len(input_data) // pool_size
    output = np.zeros(output_size)
    for i in range(output_size):
        output[i] = np.max(input_data[i * pool_size:(i + 1) * pool_size])
    return output

# Function to predict the product type
def predict(output):
    return np.argmax(output)

# Main function for training and testing
def main():
    # Initialize layers
    conv_layer = ConvLayer(FILTER_SIZE, bias=0.5)

    # Compute convolution output size
    conv_output_size = INPUT_NODES - FILTER_SIZE + 1
    # Compute pooling output size
    pool_output_size = conv_output_size // POOL_SIZE
    # Initialize Fully Connected Layer with correct input size
    fc_layer = FullyConnectedLayer(pool_output_size, OUTPUT_NODES)

    # Example training data (shortened for brevity)
    training_data = np.array([
        # [4.71, 5.63, 7.82, 1.93, 7.70, 8.37, 3.44, 2],  # Product type 2
        # [7.43, 6.79, 3.30, 3.66, 3.89, 1.91, 9.26, 3],  # Product type 3
        # [7.50, 9.06, 9.39, 4.03, 8.77, 4.16, 2.36, 3],  # Product type 3
        # [5.45, 1.20, 7.85, 8.01, 3.94, 7.62, 9.16, 0],  # Product type 0

        [4.71, 5.63, 7.82, 1.93, 7.70, 8.37, 3.44, 2],
        [7.43, 6.79, 3.30, 3.66, 3.89, 1.91, 9.26, 3],
        [7.50, 9.06, 9.39, 4.03, 8.77, 4.16, 2.36, 3],
        [5.45, 1.20, 7.85, 8.01, 3.94, 7.62, 9.16, 0],
        [3.25, 6.98, 0.58, 0.95, 5.35, 4.02, 2.04, 4],
        [0.81, 5.35, 6.43, 4.70, 7.26, 5.69, 4.03, 6],
        [4.75, 3.42, 8.79, 3.53, 7.58, 1.16, 2.71, 0],
        [2.36, 0.56, 1.04, 6.30, 8.18, 0.20, 4.96, 4],
        [7.18, 5.54, 2.38, 2.53, 9.56, 4.42, 5.30, 2],
        [9.77, 1.74, 5.07, 7.02, 7.43, 9.10, 1.78, 4],
        [2.52, 0.58, 5.71, 0.10, 1.73, 8.42, 3.13, 4],
        [8.98, 4.16, 0.40, 7.16, 4.37, 5.36, 8.59, 5],
        [0.89, 0.97, 4.08, 0.45, 5.39, 9.38, 0.82, 0],
        [1.12, 5.89, 2.18, 8.55, 4.99, 3.96, 0.73, 0],
        [4.54, 6.44, 7.60, 6.27, 4.86, 0.73, 0.37, 0],
        [4.89, 0.77, 1.00, 9.26, 6.12, 9.58, 0.81, 3],
        [0.55, 4.89, 7.46, 5.94, 4.27, 8.28, 1.10, 6],
        [4.18, 3.28, 3.94, 9.17, 7.24, 4.67, 6.67, 2],
        [1.11, 4.27, 8.05, 5.98, 5.00, 8.42, 9.82, 3],
        [9.19, 0.82, 9.15, 5.31, 0.40, 9.96, 2.32, 5],
        [4.85, 9.79, 6.89, 9.12, 8.07, 7.99, 4.51, 1],
        [1.26, 8.45, 1.41, 8.50, 3.13, 8.09, 0.28, 1],
        [2.36, 8.33, 0.22, 7.36, 6.75, 0.04, 7.26, 6],
        [0.85, 6.41, 1.24, 1.25, 6.37, 3.56, 2.21, 4],
        [3.35, 9.10, 0.35, 1.43, 7.08, 4.86, 3.67, 1],
        [3.32, 5.09, 6.85, 6.44, 3.18, 7.13, 0.68, 4],
        [5.46, 0.90, 2.90, 2.20, 0.94, 0.16, 8.14, 3],
        [6.57, 9.38, 3.04, 2.94, 2.94, 5.25, 4.17, 3],
        [4.35, 4.52, 7.72, 1.43, 9.38, 1.40, 9.78, 4],
        [6.48, 6.63, 9.13, 9.66, 3.76, 9.82, 5.20, 1],
        # Add more training data as needed
    ])

    # Training loop
    for epoch in range(EPOCHS):
        correct_predictions = 0
        for sample in training_data:
            input_data = sample[:INPUT_NODES]
            true_label = int(sample[INPUT_NODES])

            # Convolution and pooling
            conv_output = apply_convolution(input_data, conv_layer)
            pool_output = apply_pooling(conv_output, POOL_SIZE)

            # Forward pass through fully connected layer
            fc_output = fc_layer.forward(pool_output)

            # Prediction
            predicted_product = predict(fc_output)
            if predicted_product == true_label:
                correct_predictions += 1

        # Calculate and print accuracy after each epoch
        accuracy = (correct_predictions / len(training_data)) * 100
        print(f"Epoch {epoch + 1}: Accuracy: {accuracy:.2f}%")


    test_input = np.array([5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5])
    conv_output = apply_convolution(test_input, conv_layer)
    pool_output = apply_pooling(conv_output, POOL_SIZE)
    fc_output = fc_layer.forward(pool_output)
    predicted_product = predict(fc_output)
    print(f"Predicted Product Type: {predicted_product}")

if __name__ == "__main__":
    main()

Epoch 1: Accuracy: 10.00%
Epoch 2: Accuracy: 10.00%
Epoch 3: Accuracy: 10.00%
Epoch 4: Accuracy: 10.00%
Epoch 5: Accuracy: 10.00%
Epoch 6: Accuracy: 10.00%
Epoch 7: Accuracy: 10.00%
Epoch 8: Accuracy: 10.00%
Epoch 9: Accuracy: 10.00%
Epoch 10: Accuracy: 10.00%
Predicted Product Type: 5


In [6]:
import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def save_audio_features_with_custom_value(file_path, custom_value, csv_file_path):
    """
    Processes an audio file to extract MFCC, Delta, and Delta-Delta features, 
    computes their means, appends a custom value, and saves the results to a CSV.

    Args:
        file_path (str): Path to the audio file.
        custom_value (float): Custom value to append as the last column in the CSV.
        csv_file_path (str): Path to save the resulting CSV file.
    """
    # Load the audio file
    y, sr = librosa.load(file_path)

    # Extract MFCC features
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

    # Normalize the MFCCs (Min-Max Scaling)
    mfcc_normalized = (mfccs - np.min(mfccs)) / (np.max(mfccs) - np.min(mfccs))

    # Compute Delta (Δ) and Delta-Delta (ΔΔ) features
    delta_1 = librosa.feature.delta(mfcc_normalized)
    delta_2 = librosa.feature.delta(mfcc_normalized, order=2)

    # Combine static, delta, and delta-delta features
    combined_features = np.vstack([mfcc_normalized, delta_1, delta_2])

    # Compute the mean of each feature (across time axis, axis=1)
    mean_features = np.mean(combined_features, axis=1)

    # Append the custom value to the mean features as the last column
    mean_features_with_custom = np.append(mean_features, custom_value)

    # Save the updated mean features to CSV file
    df_mean = pd.DataFrame([mean_features_with_custom])
    df_mean.to_csv(csv_file_path, index=False, header=False)

    print(f"Features saved to {csv_file_path} with custom value {custom_value}.")

# Example usage
# file_path = 'audio.wav'
# custom_value = 42.42
csv_file_path = 'mean_mfcc_delta_features_with_custom_value.csv'
# save_audio_features_with_custom_value(file_path, custom_value, csv_file_path)


paths = {
    "Angry": (1, 'E:/dataset/Tess/OAF_angry/'),
    "Disgust": (2, 'E:/dataset/Tess/OAF_disgust/'),
    "Fear": (3, 'E:/dataset/Tess/OAF_Fear/'),
    "Happy": (4, 'E:/dataset/Tess/OAF_happy/'),
    "Neutral": (5, 'E:/dataset/Tess/OAF_neutral/'),
    "Pleasant Surprise": (6, 'E:/dataset/Tess/OAF_Pleasant_surprise/'),
    "Sad": (7, 'E:/dataset/Tess/OAF_sad/')
}

for emotion, (feature_number, path) in paths.items():
    if os.path.exists(path):
        print(f"{emotion} (Labels {feature_number}):")
        wav_files = [file for file in os.listdir(path) if file.endswith('.wav')]
        for file in wav_files:
            full_path = os.path.join(path, file)
            # print(f"  {full_path} -> Labels  {feature_number}")  
            save_audio_features_with_custom_value(full_path, feature_number, csv_file_path)
            

    else:
        print(f"{emotion}: Path not found!")

Angry (Labels 1):
Features saved to mean_mfcc_delta_features_with_custom_value.csv with custom value 1.
Features saved to mean_mfcc_delta_features_with_custom_value.csv with custom value 1.
Features saved to mean_mfcc_delta_features_with_custom_value.csv with custom value 1.
Features saved to mean_mfcc_delta_features_with_custom_value.csv with custom value 1.
Features saved to mean_mfcc_delta_features_with_custom_value.csv with custom value 1.
Features saved to mean_mfcc_delta_features_with_custom_value.csv with custom value 1.
Features saved to mean_mfcc_delta_features_with_custom_value.csv with custom value 1.
Features saved to mean_mfcc_delta_features_with_custom_value.csv with custom value 1.
Features saved to mean_mfcc_delta_features_with_custom_value.csv with custom value 1.
Features saved to mean_mfcc_delta_features_with_custom_value.csv with custom value 1.
Features saved to mean_mfcc_delta_features_with_custom_value.csv with custom value 1.
Features saved to mean_mfcc_delta_fe

In [ ]:
import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os  # Import os module for path checking and file operations

def save_audio_features_with_custom_value(file_path, custom_value, csv_file_path):
    """
    Processes an audio file to extract MFCC, Delta, and Delta-Delta features, 
    computes their means, appends a custom value, and saves the results to a CSV.

    Args:
        file_path (str): Path to the audio file.
        custom_value (float): Custom value to append as the last column in the CSV.
        csv_file_path (str): Path to save the resulting CSV file.
    """
    try:
        # Load the audio file
        y, sr = librosa.load(file_path)

        # Extract MFCC features
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

        # Normalize the MFCCs (Min-Max Scaling)
        mfcc_normalized = (mfccs - np.min(mfccs)) / (np.max(mfccs) - np.min(mfccs))

        # Compute Delta (Δ) and Delta-Delta (ΔΔ) features
        delta_1 = librosa.feature.delta(mfcc_normalized)
        delta_2 = librosa.feature.delta(mfcc_normalized, order=2)

        # Combine static, delta, and delta-delta features
        combined_features = np.vstack([mfcc_normalized, delta_1, delta_2])

        # Compute the mean of each feature (across time axis, axis=1)
        mean_features = np.mean(combined_features, axis=1)

        # Append the custom value to the mean features as the last column
        mean_features_with_custom = np.append(mean_features, custom_value)

        # Append the features to the CSV file
        df_mean = pd.DataFrame([mean_features_with_custom])
        if os.path.exists(csv_file_path):
            # Append to existing file
            df_mean.to_csv(csv_file_path, mode='a', index=False, header=False)
        else:
            # Create a new file with headers
            df_mean.to_csv(csv_file_path, index=False, header=False)

        print(f"Features for {file_path} saved to {csv_file_path} with custom value {custom_value}.")
    
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

# Define paths and labels
paths = {
    "Angry": (1, 'E:/dataset/Tess/OAF_angry/'),
    "Disgust": (2, 'E:/dataset/Tess/OAF_disgust/'),
    "Fear": (3, 'E:/dataset/Tess/OAF_Fear/'),
    "Happy": (4, 'E:/dataset/Tess/OAF_happy/'),
    "Neutral": (5, 'E:/dataset/Tess/OAF_neutral/'),
    "Pleasant Surprise": (6, 'E:/dataset/Tess/OAF_Pleasant_surprise/'),
    "Sad": (7, 'E:/dataset/Tess/OAF_sad/')
}

# Output CSV file path
csv_file_path = 'mean_mfcc_delta_features_with_custom_value.csv'

# Process each emotion's audio files
for emotion, (feature_number, path) in paths.items():
    if os.path.exists(path):
        print(f"Processing {emotion} (Label {feature_number}):")
        wav_files = [file for file in os.listdir(path) if file.endswith('.wav')]
        for file in wav_files:
            full_path = os.path.join(path, file)
            save_audio_features_with_custom_value(full_path, feature_number, csv_file_path)
    else:
        print(f"Path not found for {emotion}: {path}")
